In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [35]:
def GD(y, tx, initial_w, max_iters, gamma, gradient_func, loss_func):
    w = initial_w
    
    for i in range(max_iters):
        grad = gradient_func(y, tx, w)
        w = w - gamma * grad

    loss = loss_func(y, tx, w)
    return w, loss

def GD_reg(y, tx, initial_w, max_iters, gamma, gradient_func, loss_func, lambda_):
    w = initial_w
    
    for i in range(max_iters):
        grad = gradient_func(y, tx, w, lambda_)
        w = w - gamma * grad

    loss = loss_func(y, tx, w)
    return w, loss

In [24]:
def get_random_sample(y, tx):
    random_sample_index = np.random.randint(len(y))
    y_sample  = y [random_sample_index]
    tx_sample = tx[random_sample_index]
    return y_sample, tx_sample

def SGD(y, tx, initial_w, max_iters, gamma, gradient_func, loss_func):
    w = initial_w
    
    for n_iter in range(max_iters):
        y_sample, tx_sample = get_random_sample(y, tx)
        grad = gradient_func(y_sample, tx_sample, w)
        w = w - gamma * grad
        
    loss = loss_func(y, tx, w)
    return w, loss

# Least Squares

In [25]:
def MSE(y, tx, w):
    e = y - tx.dot(w)
    return np.mean(e**2)

def least_squares_gradient(y, tx, w):
    e = y - tx.dot(w)
    return -tx.T.dot(e)/y.size

def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    w, loss = GD(y, tx, initial_w, max_iters, gamma, least_squares_gradient, MSE)
    return w, loss

def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    w, loss = SGD(y, tx, initial_w, max_iters, gamma, least_squares_gradient, MSE)
    return w, loss

def least_squares(y : np.array, tx : np.array):
    Q, R = np.linalg.qr(tx)
    w = np.linalg.solve(R, Q.T.dot(y))
    loss = MSE(y, tx, w)
    
    return w, loss

In [30]:
def ridge_regression(y : np.array, tx: np.array , lambda_):
    D = tx.shape[1]
    lambda_I = np.eye(D) * np.sqrt(2*len(y)*lambda_)
    tx_expended = np.append(tx, lambda_I, axis=0)
    y_expended  = np.append(y, np.zeros(D))
    
    Q, R = np.linalg.qr(tx_expended)
    w = np.linalg.solve(R, Q.T.dot(y_expended))
    loss = MSE(y, tx, w)
    
    return w, loss

# Logistic Regression

# Logistic Regression

In [37]:
def sigmoid(t):
    exp_t = np.exp(t)
    return exp_t / (1 + exp_t)

def logistic_loss(y, tx, w):
    xtw = tx.dot(w)
    loss = np.sum(np.log(1 + np.exp(xtw))) - y.T.dot(xtw)
    return np.squeeze(loss)

def logistic_gradient(y, tx, w):
    return tx.T.dot(sigmoid(tx.dot(w)) - y)

def reg_logistic_gradient(y, tx, w, lambda_):
    return logistic_gradient(y, tx, w) + 2 * lambda_ * w

In [38]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    w, loss = GD(y, tx, initial_w, max_iters, gamma, logistic_gradient, logistic_loss)
    return w, loss

def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    w, loss = GD_reg(y, tx, initial_w, max_iters, gamma, reg_logistic_gradient, logistic_loss, lambda_)
    return w, loss

# Sandbox for testing

In [33]:
x = [[12,16,71,99,45,27,80,58,4,50],
     [35,78,73,3,55,43,56,98,32,40]]
x = (x-np.mean(x, axis=1).reshape(2,1))/np.std(x, axis=1).reshape(2,1)
y = [56,22,37,78,83,55,70,94,12,40]
x = np.array(x).T
tx = np.insert(x, 0, 1, axis=1)
y = np.array(y)
linreg = LinearRegression().fit(x,y)
initial_w = np.array([0,0,0])
w = initial_w


print(np.append(np.array(linreg.intercept_), linreg.coef_))

[54.7        15.08492696  3.55532095]


### Testing for least Squares GD

In [36]:
least_squares_GD(y, tx, initial_w, 100, 0.1)

(array([54.69854709, 15.08425522,  3.55477245]), 423.1618830220783)

### Testing for least Squares SGD

In [13]:
least_squares_SGD(y, tx, initial_w, 100000, 0.0001)

(array([54.77003147, 15.21760936,  3.704129  ]), 423.20331285012253)

### Testing least squares

In [14]:
least_squares(y, tx)

(array([54.7       , 15.08492696,  3.55532095]), 423.16188021914616)

### Testing ridge regression

In [17]:
ridge_regression(y, tx , lambda_=0.001)

(array([54.59081836, 15.05403516,  3.54571034]), 423.1747990944212)